In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
import json
import pandas as pd
import time
from extracted_questions import Physics, Chemistry, Mathematics ,PhysicsA, ChemistryA, MathematicsA 
import re


75
75


In [2]:
physics_topics = ["Vectors","Types of Forces","Newton’s Laws of Motion","Friction","Basics of calculus","1 D Kinematics","2 D Kinematics","Circular Motion","Work, Power and Energy","Center of Mass","Conservation of Momentum","Rotation","Fluid Statics and Dynamics","Properties of Solids","Surface Tension and Viscosity","Simple Harmonic Motion","Combination of SHM","Damping and Resonance","Waves on a string","Sound Waves","Light Waves and Interference","Ray Optics","Thermal Expansion and Calorimetry","Kinetic Theory of Gases","1st Law of Thermodynamics & specific heats","Heat transfer","Electrostatics","Dielectrics and Capacitors","Current Electricity","Magnetism","Electro Magnetic Induction","Alternating Current","Modern Physics","Gravitation"]

maths_topics = ["Number System", "Factors and Indices","Sets","Progressions and Series","Quadratic Equations & Theory of Equations","Logarithms","Trigonometric Identities and Equations","Determinants and Matrices","Straight Lines and Pair of Lines","Circles","Permutations and Combinations","Binomial Theorem","Complex Numbers","Solution of Triangles","Functions","Inverse Trigonometry","Limits and Continuity","Derivatives and Method of Differentiation","Application of Derivatives","Indefinite Integration","Definite Integration","Area Under the Curves","Differential Equations","Vectors and 3D Geometry","Probability","Parabola","Ellipse","Hyperbola"
]

chemistry_topics =["Periodic Table","Chemical Bonding","Atomic Structure","Mole Concept","Gaseous State","Thermodynamics","Thermochemistry","Chemical Equilibrium","Ionic Equilibrium","Redox Reactions","s-block Elements","Hydrogen and Related Compounds","Nomenclature","Solid State","Boron Family","Structural and Geometrical Isomerism","Liquid Solutions and Colligative Properties","Chemical Kinetics","Surface Chemistry","Carbon Family","Metallurgy","d-block elements","f-block elements","General Organic Chemistry","Electrochemistry","Reaction Mechanisms","Tautomerism","Optical Isomerism","Alkanes","Alkenes","Alkynes","Arenes","Alkyl and Aryl Halides","Alcohols, Phenols and Ethers","Aldehydes and Ketones","Nitrogen Family","Amines","Carboxylic Acid and Derivatives","Qualitative Analysis","Coordination compounds","Biomolecules and Polymers","16, 17 and 18 family","Practical Organic Chemistry"
]
maths_questions = Mathematics
chemistry_questions = Chemistry
physics_questions = Physics
maths_answers = MathematicsA
chemistry_answers = ChemistryA
physics_answers = PhysicsA




In [3]:

accurate_prompt = strict_prompt = ChatPromptTemplate.from_template("""
SYSTEM:
You are a classification assistant. You MUST output ONLY a single valid JSON object and NOTHING else.
Use only the exact category names listed below (exact spelling). Do not invent or alter category names.

Categories:
{maths_topics}

RULES (must follow exactly):
1) Output only ONE JSON object. No explanation, no backticks, no extra text.
2) Use ONLY keys from the categories above (exact spellings).
3) The numeric values must be floats and their sum must be exactly 100.0.
4) If a question clearly belongs to a single category, give that category 100.0.
5) If ambiguous, split percentages but still sum to 100.0.
6) classify based on the answer, not just the question.
7) properly read and understand the question and answer before classifying.
8) only add non zero results to the JSON object.

### EXAMPLE ###
QUESTION: What is the integral of 2x dx?
OUTPUT (ONLY the valid JSON object):
{{"Calculus": 100.0}}

QUESTION: {maths_questions}

OUTPUT (ONLY the valid JSON object):
""")


In [4]:
# model = ChatOllama(model="llama3:8b", temperature=0)
# model = ChatOllama(model="deepseek-r1:8b", temperature=0)
# model = ChatOllama(model="qwen3:8b", temperature=0)
model = ChatOllama(model="magistral:latest", temperature=0)

In [5]:
def get_topic_vector(question,topic_string = maths_topics):
    """Get the topic vector for a single question"""
    try:
        chain = accurate_prompt | model
        response = chain.invoke({"maths_questions": question ,"maths_topics":topic_string})
        response_text = response.content.strip()
        json_start = response_text.find('{')
        json_end = response_text.rfind('}') + 1
        json_str = response_text[json_start:json_end]
        
        return json.loads(json_str)
    except Exception as e:
        print(f"Error: {e}")
        return {"Error": 100.0}
    

In [6]:

all_results = []
for i, q in enumerate(maths_questions):
    print(f"Processing question {i+1}...")
    if i%10 == 0 and i != 0:
        time.sleep(2)  # Pause for 2 seconds every 10 questions to avoid rate limits
    if i == 10:
        break
    topic_data = get_topic_vector(q)
    all_results.append({
        "question_id": i+1,
        "question": q,
        "topic_vector": topic_data
    })

Processing question 1...


KeyboardInterrupt: 

In [ ]:
all_results_df_maths = pd.DataFrame(all_results)
all_results_df_maths.head(5)
# all_results_df_maths.to_excel("qwen/maths_ollama.xlsx", index=False)


In [ ]:
all_results

[{'question_id': 1,
  'question': "Q.1 If y = f(x) makes positive intercepts of 2 and 0 units on the x and y axes respectively and encloses an area of 3/4 square units with the axes, then ∫₀² xf'(x)dx is (A) 3/2 (B) 1 (C) 5/4 (D) -3/4",
  'topic_vector': {'Functions': 100.0}},
 {'question_id': 2,
  'question': 'Q.2 The value of ∫₀¹⁰π |sin(x)|dx is (A) 20 (B) 8 (C) 10 (D) 18',
  'topic_vector': {'Trigonometric Identities and Equations': 100.0}},
 {'question_id': 3,
  'question': 'Q.3 Let f(x) = 7tan⁸(x) + 7tan⁶(x) - 3tan⁴(x) - 3tan²(x). If I₁ = ∫₀^(π/4) f(x)dx and I₂ = ∫₀^(π/4) xf(x)dx, then 7I₁ + 12I₂ is equal to: (A) 2π (B) 1 (C) π (D) 2',
  'topic_vector': {'Quadratic Equations & Theory of Equations': 100.0}},
 {'question_id': 4,
  'question': 'Q.4 If f(a+b-x) = f(x), then ∫ₐᵇ xf(x)dx is equal to (A) (a+b)/2 ∫ₐᵇ f(a+b+x)dx (B) (a+b)/2 ∫ₐᵇ f(b-x)dx (C) (a+b)/2 ∫ₐᵇ f(x)dx (D) (b-a)/2 ∫ₐᵇ f(x)dx',
  'topic_vector': {'Derivatives and Method of Differentiation': 100.0}},
 {'question_id': 

In [ ]:
[{'question_id': 1,
  'question': 'Q.1 A Capacitor stores 60μC charge when connected across a battery. When the gap between the plates is filled with a dielectric, a charge of 120μC flows through the battery, the dielectric constant of the material inserted is (A) 5 (B) 2 (C) 3 (D) 4',
  'topic_vector': {'Properties of Solids': 100.0}},
 {'question_id': 2,
  'question': 'Q.2 A capacitor is connected to a battery. The force of attraction between the plates when the separation between them is halved is (A) Remains the same (B) Becomes two times (C) Becomes eight times (D) Becomes four times',
  'topic_vector': {'Dielectrics and Capacitors': 100.0}},
 {'question_id': 3,
  'question': 'Q.3 A flat air capacitor C consists of two large plates that are close to each other. Initially, one of the plates was not charged, while the other had charge Q. If entire space between the plates is filled with a slab of finite electrical resistance, estimate total amount of energy lost in the slab. (A) Q²/2C (B) Q²/8C (C) 3Q²/8C (D) 7Q²/8C',
  'topic_vector': {'Electrostatics': 100.0}},
 {'question_id': 4,
  'question': 'Q.4 In the circuit shown, after the switch is shifted to position 2 the heat generated in 50 Ω resistance is 7.5 J. Find the EMF of the cell.  (A) 200 V (B) 460 V (C) 400 V (D) 180 V',
  'topic_vector': {'Electrostatics': 100.0}},
 {'question_id': 5,
  'question': 'Q.5 Consider the loop PQRSP, carrying clockwise current i, shown in the figure. The magnitude of magnetic field at the centre O of the curved portion is  (A) μ₀i/2πr[π-ϕ+tanϕ] (B) μ₀i/2πr (C) 0 (D) μ₀i/2πr[π-ϕ-tanϕ]',
  'topic_vector': {'Electro Magnetic Induction': 100.0}},
 {'question_id': 6,
  'question': 'Q.6 The straight long conductors AOB and COD are perpendicular to each other and carry current i₁ and i₂. The magnitude of the magnetic induction at point P at a distance a from the point O in a direction perpendicular to the plane ACBD is  (A) μ₀/2πa(i₁+i₂) (B) μ₀/2πa(i₁-i₂) (C) μ₀/2πa(i₁²+i₂²)^(1/2) (D) μ₀/2πa(i₁i₂/(i₁+i₂))',
  'topic_vector': {'Simple Harmonic Motion': 100.0}},
 {'question_id': 7,
  'question': 'Q.7 A wire of length L carrying current I is bent into a circle of one turn. The field at the center of the coil is B₁. A similar wire of length L carrying current I is bent into a square of one turn. The field at its center is B₂. Then (A) B₁>B₂ (B) B₁=B₂ (C) B₁/B₂=2 (D) B₁<B₂',
  'topic_vector': {'Magnetism': 100.0}},
 {'question_id': 8,
  'question': 'Q.8 A particle of mass m and charge q is in an electric and magnetic field given by E=2i+3j, B=4j+6k. The charged particle shifted from the origin to the point (1,1) along a straight line path. The magnitude of total work done is. (A) 0.5q (B) 0.25q (C) 5q (D) 2.5q',
  'topic_vector': {'Electrostatics': 100.0}},
 {'question_id': 9,
  'question': 'Q.9 Find the magnitude of the magnetic induction B of a magnetic field generated by a system of thin conductors along which a current i is flowing at a point A(0,R,0), that is the centre of a circular conductor of radius R. The ring is in yz plane.  (A) B=μ₀i/4πR(2π²-2π+1) (B) B=μ₀i/4πR * sqrt(2π²-2π+1) (C) B=μ₀i/2πR(2π²-2π+1) (D) None of these',
  'topic_vector': {'Modern Physics': 100.0}},
 {'question_id': 10,
  'question': 'Q.10 For c=2a if, the magnetic field at point P will be zero when  (A) a=b (B) a=3/5b (C) a=5/3b (D) a=1/3b',
  'topic_vector': {'Magnetism': 100.0}},
 {'question_id': 11,
  'question': 'Q.11 Find the magnitude and direction of magnetic field at point P due to the current carrying wire as shown in figure.  (A) μ₀i/4πa(√3-1) (B) μ₀i/8πa(√3-1) (C) μ₀i/8πa(π-√3) (D) μ₀i/4πa(π-√3)',
  'topic_vector': {'Magnetism': 100.0}},
 {'question_id': 12,
  'question': 'Q.12 A particle of mass m and charge q is thrown from origin at t=0 with velocity 2i+3j+4k units in a region with uniform magnetic field B=2i units. After time t=πm/qB, an electric field E is switched on such that particle moves on a straight line with constant speed. E may be: (A) 5k-10j units (B) -6k-9j units (C) -6k+8j units (D) 6k+8j units',
  'topic_vector': {'Types of Forces': 100.0}},
 {'question_id': 13,
  'question': 'Q.13 Figure shows a vertical force F that is applied tangentially to a uniform cylinder of weight W. The coefficient of static friction between the cylinder and all surfaces is 0.5. Find in terms of W, the maximum force that can be applied without causing the cylinder to rotate.  (A) W/4 (B) 3W/4 (C) W/8 (D) 3W/8',
  'topic_vector': {'1 D Kinematics': 0,
   '2 D Kinematics': 0,
   'Circular Motion': 100.0}},
 {'question_id': 14,
  'question': 'Q.14 A Particle having mass 2 KG is moving along straight line 3x+4y=5 with speed 8 m/s. Find angular momentum of the particle about origin. x and y are in metres. (A) 16 kg m²/s (B) 80 kg m/s² (C) 16/5 kg m²/s (D) 8 kg m/s²',
  'topic_vector': {'Vectors': 100.0}},
 {'question_id': 15,
  'question': 'Q.15 Figure shows a rough track, a portion of which is in form of a cylinder of radius R. With what minimum linear speed should a sphere of radius r be set rolling on the horizontal part so that it completely goes round the circle on the cylindrical part.  (A) sqrt(5g(R-r)) (B) sqrt(22/7g(R-r)) (C) sqrt(17/7g(R-r)) (D) sqrt(27/7g(R-r))',
  'topic_vector': {'Work, Power and Energy': 100.0}},
 {'question_id': 16,
  'question': 'Q.16 A cylinder rolls up an inclined plane, reaches some height and then rolls down (without slipping throughout these motions). The directions of the frictional force acting on the cylinder are: (A) Up the incline while ascending and down the incline while descending (B) Up the incline while ascending as well as descending (C) Down the incline while ascending and up the incline while descending (D) Down the incline while ascending as well as descending.',
  'topic_vector': {'Newton’s Laws of Motion': 100.0}},
 {'question_id': 17,
  'question': "Q.17 Water flows out of a big tank along a tube bent at right angles; the inside radius of the tube is equal to r. The length of the horizontal section of the tube is equal to l. The water flow rate is Q. Find the moment of reaction forces of flowing water, acting on the tube's walls, relative to the point O. ρ= density of water.  (A) 2ρQ²l/πr² (B) ρQ²l/πr² (C) ρQ²l/2al² (D) None of these.",
  'topic_vector': {'Types of Forces': 100.0}},
 {'question_id': 18,
  'question': "Q.18 A vessel with liquid of density 'ρ' falls with acceleration g. Find the pressure in the vessel as a function of depth 'h'. Given P₀= atmospheric pressure. (A) P₀+ρgh (B) P₀-ρgh (C) ρgh (D) P₀",
  'topic_vector': {'Types of Forces': 100.0}},
 {'question_id': 19,
  'question': 'Q.19 A piece of ice floats in a vessel with water above which a layer of lighter oil is poured. What happens to the level of oil and level of water in the vessel when the ice completely melts. (A) level of both oil and water increases. (B) level of oil increases and level of water remains same. (C) level of oil decreases and level of water increases. (D) level of water increases but level of oil remains same.',
  'topic_vector': {'Thermal Expansion and Calorimetry': 100.0}},
 {'question_id': 20,
  'question': "Q.20 Incompressible fluid of negligible viscosity is pumped at a volume flow rate Q through a small hole into the narrow gap between two clearly spaced parallel plates as shown. The liquid flowing away from the hole only has radial motion. Find the speed as a function of radius from the hole. Separation between the plates is 'd'.  (A) Q/πd² (B) Q/πr² (C) Q/πrd (D) Q/2πrd",
  'topic_vector': {'Fluid Statics and Dynamics': 100.0}},
 {'question_id': 21,
  'question': 'Q.21 A parallel plate capacitor which is disconnected is at a certain potential difference. When a 3 mm thick dielectric slab is introduced between the plates, the plate separation has to be increased by 2 mm in order to maintain the same potential difference between the plates. Find the dielectric constant of the slab.',
  'topic_vector': {'Dielectrics and Capacitors': 100.0}},
 {'question_id': 22,
  'question': 'Q.22 A slender rod of length L forming an angle theta with the horizontal strikes, a frictionless floor at A with its centre of mass velocity Vc and no angular velocity. Assumed that the impact at A is perfectly inelastic, find the angular velocity of the rod immediately after the impact. Given: L=1 m, Vc=5√3 m/s, θ=30°',
  'topic_vector': {'1 D Kinematics': 0.0,
   '2 D Kinematics': 0.0,
   'Circular Motion': 100.0}},
 {'question_id': 23,
  'question': 'Q.23 Find the magnetic field due to current i flowing in an elliptical loop at its focus. The equation of ellipse (in polar coordinates as shown) is l/r=(1+e cosθ). Here e is eccentricity which is a constant. Take l=50 cm, e=0.8, i=2A, if your answer is nπ×10⁻⁷ T, fill n in OMR sheet. ',
  'topic_vector': {'Magnetism': 100.0}},
 {'question_id': 24,
  'question': "Q.24 A uniform conducting wire ABC has a mass of 10 g. A current of 2A flows through it. The wire is kept in a uniform magnetic field B=2T. The acceleration of the wire is 2x. Find 'x'? ",
  'topic_vector': {'1 D Kinematics': 100.0}},
 {'question_id': 25,
  'question': "Q.25 Two parallel long wires carry currents i₁ & i₂. When the currents are in the same direction, the magnetic field midway between the wires is 10μT. When the direction of i₂ is reversed, it becomes 40μT. If the ratio i₁/i₂ = a/b where 'a' and 'b' are the smallest integers, find (a+b).",
  'topic_vector': {'Magnetism': 100.0}}]

[{'question_id': 1,
  'question': 'Q.1 A Capacitor stores 60μC charge when connected across a battery. When the gap between the plates is filled with a dielectric, a charge of 120μC flows through the battery, the dielectric constant of the material inserted is (A) 5 (B) 2 (C) 3 (D) 4',
  'topic_vector': {'Properties of Solids': 100.0}},
 {'question_id': 2,
  'question': 'Q.2 A capacitor is connected to a battery. The force of attraction between the plates when the separation between them is halved is (A) Remains the same (B) Becomes two times (C) Becomes eight times (D) Becomes four times',
  'topic_vector': {'Dielectrics and Capacitors': 100.0}},
 {'question_id': 3,
  'question': 'Q.3 A flat air capacitor C consists of two large plates that are close to each other. Initially, one of the plates was not charged, while the other had charge Q. If entire space between the plates is filled with a slab of finite electrical resistance, estimate total amount of energy lost in the slab. (A) Q²/